In [1]:
%load_ext tensorboard 

In [2]:
!rm -rf ./logs/ 

In [3]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

2023-02-20 15:44:42.400604: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 [==============================] - 1s 0us/step


In [6]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([256, 512]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.5, 0.6))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam','sgd','rmsprop']))
HP_L2 = hp.HParam('l2 regularizer', hp.RealInterval(.001,.01))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER,HP_L2],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

2023-02-20 15:45:56.281614: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
def train_test_model(hparams):
  model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(hparams[HP_NUM_UNITS], kernel_regularizer=tf.keras.regularizers.l2(0.001), activation=tf.nn.relu),
    tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax),
  ])
  model.compile(
      optimizer=hparams[HP_OPTIMIZER],
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'],
  )

  model.fit(x_train, y_train, epochs=2) 
  _, accuracy = model.evaluate(x_test, y_test)
  return accuracy

In [8]:

def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=2)

In [9]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
    for l2 in (HP_L2.domain.min_value, HP_L2.domain.max_value):
      for optimizer in HP_OPTIMIZER.domain.values:
        hparams = {
            HP_NUM_UNITS: num_units,
            HP_DROPOUT: dropout_rate,
            HP_L2: l2,
            HP_OPTIMIZER: optimizer,
        }
        run_name = "run-%d" % session_num
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('logs/hparam_tuning/' + run_name, hparams)
        session_num += 1

--- Starting trial: run-0
{'num_units': 256, 'dropout': 0.5, 'l2 regularizer': 0.001, 'optimizer': 'adam'}
Epoch 1/2
1875/1875 [==============================] - 3s 1ms/step - loss: 0.4978 - accuracy: 0.9003
Epoch 2/2
313/313 [==============================] - 0s 806us/step - loss: 0.2487 - accuracy: 0.9599
--- Starting trial: run-1
{'num_units': 256, 'dropout': 0.5, 'l2 regularizer': 0.001, 'optimizer': 'rmsprop'}
Epoch 1/2
1875/1875 [==============================] - 3s 2ms/step - loss: 0.4748 - accuracy: 0.8971
Epoch 2/2
313/313 [==============================] - 0s 773us/step - loss: 0.2264 - accuracy: 0.9552
--- Starting trial: run-2
{'num_units': 256, 'dropout': 0.5, 'l2 regularizer': 0.001, 'optimizer': 'sgd'}
Epoch 1/2
1875/1875 [==============================] - 2s 1ms/step - loss: 1.1388 - accuracy: 0.7818
Epoch 2/2
313/313 [==============================] - 0s 807us/step - loss: 0.6383 - accuracy: 0.9207
--- Starting trial: run-3
{'num_units': 256, 'dropout': 0.5, 'l2 regula

In [11]:
%tensorboard --logdir logs/hparam_tuning --port 6050 
